In [17]:
import torch
import torch.nn as nn

In [18]:
torch.cuda.is_available()

True

# Patch embedding

In [19]:
# B, C, H, W
img = torch.rand((100, 3, 224, 224))
print(img.shape)

torch.Size([100, 3, 224, 224])


In [20]:
img_size = (224, 224)
patch_size = (16, 16)
grid_size = (img_size[0]//patch_size[0], img_size[1]//patch_size[1])
num_patches = grid_size[0] * grid_size[1]
proj = nn.Conv2d(3, 768, kernel_size=patch_size, stride=patch_size)
norm = nn.Identity()

In [21]:
B, C, H, W = img.shape
x = proj(img)
print(x.shape)
x = x.flatten(2)
print(x.shape)
x = x.transpose(1,2)
print(x.shape)

torch.Size([100, 768, 14, 14])
torch.Size([100, 768, 196])
torch.Size([100, 196, 768])


# Attention

In [22]:
num_heads = 16
dim = 768
head_dim = dim//num_heads
qkv = nn.Linear(dim, dim*3, bias = False)
scale = head_dim ** -0.5
attn_drop = 0.2
proj1 = nn.Linear(dim, dim)
proj_drop = nn.Dropout(0.2)

In [23]:
B, N, C = x.shape
x = qkv(x)
print(x.shape)
x = x.reshape(B, N, 3, num_heads, C//num_heads)
print(x.shape)
x = x.permute(2, 0, 3, 1, 4)
print("[3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]")
print(x.shape)

torch.Size([100, 196, 2304])
torch.Size([100, 196, 3, 16, 48])
[3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]
torch.Size([3, 100, 16, 196, 48])
